In [11]:
import numpy as np
import cvxpy as cp

from problems.data_generation import generate_problem_data
from problems.problem_definition import BilevelProblem
from algorithms.barrier_blo_box import BarrierBLO
from algorithms.blocc import BLOCC
from algorithms.implicit_gradient_descent import IGD

n = 60
seed = 8

data = generate_problem_data(n, seed)
problem = BilevelProblem(data)

x_init = np.zeros(n)
y_init = np.zeros(n)
y_g_init = np.zeros(n)
y_F_init = np.zeros(n)
mu_g_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
mu_F_init = np.zeros(problem.num_constraints_h1 + problem.num_constraints_h2)
hparams = {
        'barrier_blo': {
            'M': 0.001,
            't': 0.01,
            'alpha_x': 0.002,
            'alpha_y': 0.1,
            'beta_y': 1,
            'epsilon_x': 0.1,
            'epsilon_y': 0.01,
            'inner_max_iters': 1000000,
            'outer_max_iters': 100000
        },
        'blocc': {
            'gamma': 10,
            'alpha_x': 0.001,
            'alpha_g_y': 0.001,
            'alpha_F_y': 0.001,
            'beta_g_y': 0.001,
            'beta_F_y': 0.001,
            'epsilon_x': 0.1,
            'epsilon_inner_y_g': 0.01,
            'epsilon_outer_y_g': 0.01,
            'epsilon_inner_y_F': 0.01,
            'epsilon_outer_y_F': 0.01,
            'maxmin_g_outer_max_iters': 5000,
            'maxmin_F_outer_max_iters': 5000,
            'maxmin_g_inner_max_iters': 5000,
            'maxmin_F_inner_max_iters': 5000,
            'main_max_iters': 5000 
        },
        'IGD': {
            'M': 1e-3,
            't': 1e-3,
            'alpha_x': 0.001,
            'alpha_y': 0.001,
            'epsilon_x': 1e-3,
            'epsilon_y': 1e-3,
            'inner_max_iters': 1000,
            'outer_max_iters': 1000
        }
    }

# BLOCC Algorithm

In [9]:
blocc_algo = BLOCC(problem, hparams)


x_opt_blocc, y_opt_blocc, history = blocc_algo.blocc(x_init, y_g_init, y_F_init, mu_g_init, mu_F_init)
print(problem.f(x_opt_blocc, y_opt_blocc))



Main iteration 1
Inner iter for L_g=0, Projected Gradient norm w.r.t y of L_g=77.81704153080216
Inner iter for L_g=1, Projected Gradient norm w.r.t y of L_g=73.1800657659752
Inner iter for L_g=2, Projected Gradient norm w.r.t y of L_g=68.82482146033996
Inner iter for L_g=3, Projected Gradient norm w.r.t y of L_g=64.73385095526986
Inner iter for L_g=4, Projected Gradient norm w.r.t y of L_g=60.89079941385908
Inner iter for L_g=5, Projected Gradient norm w.r.t y of L_g=57.28034387644168
Inner iter for L_g=6, Projected Gradient norm w.r.t y of L_g=53.88812695587442
Inner iter for L_g=7, Projected Gradient norm w.r.t y of L_g=50.70069486474364
Inner iter for L_g=8, Projected Gradient norm w.r.t y of L_g=47.70543948732987
Inner iter for L_g=9, Projected Gradient norm w.r.t y of L_g=44.89054422843475
Inner iter for L_g=10, Projected Gradient norm w.r.t y of L_g=42.2449333891392
Inner iter for L_g=11, Projected Gradient norm w.r.t y of L_g=39.75822483631061
Inner iter for L_g=12, Projected Gr

# BLOCC Feasibility Check

In [10]:
barrier_algo = BarrierBLO(problem, hparams)

h1_values = [problem.h_1(x_opt_blocc, y_opt_blocc, i) for i in range(problem.num_constraints_h1)]
h2_values = [problem.h_2(x_opt_blocc, y_opt_blocc, i) for i in range(problem.num_constraints_h2)]

y_original_opt_blocc = barrier_algo.Interior_inner_loop(x_opt_blocc, y_opt_blocc)
print(f"f_theoretical_opt_result={problem.f(x_opt_blocc, y_original_opt_blocc)}, f_blocc_numerical_result={problem.f(x_opt_blocc, y_opt_blocc)}")

print(f"g_theoretical_opt_result={problem.g(x_opt_blocc, y_original_opt_blocc)}, g_blocc_numerical_result={problem.g(x_opt_blocc, y_opt_blocc)}")

for i, h_val in enumerate(h1_values):
    print(f"h_1[{i}] = {h_val}")

for i, h_val in enumerate(h2_values):
    print(f"h_2[{i}] = {h_val}")

f_theoretical_opt_result=55.38543215230454, f_blocc_numerical_result=26.027630044534384
g_theoretical_opt_result=-47.40757642275439, g_blocc_numerical_result=-46.07475015922486
h_1[0] = -1.1798194350766176
h_1[1] = -0.4369119805949942
h_1[2] = 0.0
h_1[3] = 2.220446049250313e-16
h_1[4] = -0.46454202593872124
h_1[5] = -0.11499421898496606
h_1[6] = -0.6510333163820454
h_1[7] = -1.1102230246251565e-16
h_1[8] = -1.0013708462002409
h_1[9] = -1.0132487134322172
h_1[10] = -1.1102230246251565e-16
h_1[11] = -1.1764215892939442
h_1[12] = -0.14768637223231998
h_1[13] = -1.3158717610835358
h_1[14] = -1.3470922817435378
h_1[15] = -1.1102230246251565e-16
h_1[16] = -0.47619235374019375
h_1[17] = -0.014587158530667332
h_1[18] = -0.6621628796536442
h_1[19] = -1.6653345369377348e-16
h_2[0] = -1.0852374205016784
h_2[1] = -1.0098944212747674
h_2[2] = -0.9785418426640754
h_2[3] = -1.061388066461694
h_2[4] = -1.084057836541696
h_2[5] = -0.8579512162707544
h_2[6] = -1.0024349222274238
h_2[7] = -1.138338280871

# Implicit Gradient Descent Algorithm

In [49]:
IGD_algo = IGD(problem, hparams)

x_opt_IGD, y_opt_IGD, history = IGD_algo.upper_loop(x_init, y_init)

print("Optimized x:", x_opt_IGD)
print("Optimized y:", y_opt_IGD)

h1_values = [problem.h_1(x_opt_IGD, y_opt_IGD, i) for i in range(problem.num_constraints_h1)]

print("\nValues of h_1 constraints at the optimal point:")
for i, h_val in enumerate(h1_values):
    print(f"h_1[{i}] = {h_val}")

times = [h['time'] for h in history]
grad_norms = [h['grad_norm'] for h in history]

Outer iteration 1
Inner loop converged at iteration 200
f(x, y) = 95.45037851360806, grad_norm = 76.59673418115331
Outer iteration 2
Inner loop converged at iteration 112
f(x, y) = 91.19500851740221, grad_norm = 67.2257525452191
Outer iteration 3
Inner loop converged at iteration 110
f(x, y) = 87.9160198480594, grad_norm = 59.00630049516197
Outer iteration 4
Inner loop converged at iteration 107
f(x, y) = 85.39015297241798, grad_norm = 51.796437571852486
Outer iteration 5
Inner loop converged at iteration 105
f(x, y) = 83.44417540097366, grad_norm = 45.471557030758525
Outer iteration 6
Inner loop converged at iteration 102
f(x, y) = 81.94475905747844, grad_norm = 39.922553604489096
Outer iteration 7
Inner loop converged at iteration 100
f(x, y) = 80.7892951506114, grad_norm = 35.053784045327106
Outer iteration 8
Inner loop converged at iteration 98
f(x, y) = 79.8987827656407, grad_norm = 30.781487703596458
Outer iteration 9
Inner loop converged at iteration 95
f(x, y) = 79.212391899265

# Implicit Gradient Descent Feasibility Check

In [50]:
barrier_algo = BarrierBLO(problem, hparams)

h1_values = [problem.h_1(x_opt_IGD, y_opt_IGD, i) for i in range(problem.num_constraints_h1)]
h2_values = [problem.h_2(x_opt_IGD, y_opt_IGD, i) for i in range(problem.num_constraints_h2)]

y_original_opt_IGD = barrier_algo.Interior_inner_loop(x_opt_IGD, y_opt_IGD)
print(f"f_theoretical_opt_result={problem.f(x_opt_IGD, y_original_opt_IGD)}, f_IGD_numerical_result={problem.f(x_opt_IGD, y_opt_IGD)}")

print(f"g_theoretical_opt_result={problem.g(x_opt_IGD, y_original_opt_IGD)}, g_IGD_numerical_result={problem.g(x_opt_IGD, y_opt_IGD)}")

for i, h_val in enumerate(h1_values):
    print(f"h_1[{i}] = {h_val}")

for i, h_val in enumerate(h2_values):
    print(f"h_2[{i}] = {h_val}")

f_theoretical_opt_result=59.04726374415966, f_IGD_numerical_result=76.90380080947456
g_theoretical_opt_result=-47.905074255466786, g_IGD_numerical_result=-51.811093935401786
h_1[0] = -1.818606806017621
h_1[1] = -0.6332653285456722
h_1[2] = 0.692060467876725
h_1[3] = -0.19201660481909777
h_1[4] = -0.600298040133991
h_1[5] = -0.2737396315591377
h_1[6] = -1.012366935480417
h_1[7] = 0.4450296606186358
h_1[8] = -1.4287421110645298
h_1[9] = -0.8173446049192401
h_1[10] = 1.0874902560609827
h_1[11] = -1.4789022155643556
h_1[12] = 0.10827124470942368
h_1[13] = -1.786263928772887
h_1[14] = -0.8903993418731315
h_1[15] = -0.07504978497512865
h_1[16] = -0.3015547409733911
h_1[17] = -0.21462543837571568
h_1[18] = -0.693290361744195
h_1[19] = 0.5416734352752851
h_2[0] = -1.0929089554460576
h_2[1] = -0.9654870816489937
h_2[2] = -1.0237375559765334
h_2[3] = -1.0192583555543342
h_2[4] = -1.044948936664087
h_2[5] = -0.877464459734365
h_2[6] = -1.0860014094637798
h_2[7] = -1.2018995373962
h_2[8] = -1.4768

# BFBM Algorithm

In [6]:
barrier_algo = BarrierBLO(problem, hparams)

# y_proj = barrier_algo.project_to_constraints(x_init, y_init)

# h = problem.h_1(x_init, y_proj, 0)

# print(y_proj)

# print(h)



x_opt_barrier, y_opt_barrier, history = barrier_algo.upper_loop(x_init, y_init)

print("Optimized x:", x_opt_barrier)
print("Optimized y:", y_opt_barrier)

h1_values = [problem.h_1(x_opt_barrier, y_opt_barrier, i) for i in range(problem.num_constraints_h1)]

print("\nValues of h_1 constraints at the optimal point:")
for i, h_val in enumerate(h1_values):
    print(f"h_1[{i}] = {h_val}")

times = [h['time'] for h in history]
grad_norms = [h['grad_norm'] for h in history]

Outer iteration 1
Inner loop converged at iteration 86
f(x, y) = 90.49973731529965, grad_norm of hyperfunction= 77.86208771566764
Outer iteration 2
Inner loop converged at iteration 66
f(x, y) = 105.03488086901457, grad_norm of hyperfunction= 516.5132229121291
Outer iteration 3
Inner loop converged at iteration 95
f(x, y) = 39.34324259400669, grad_norm of hyperfunction= 154.568445797518
Outer iteration 4
Inner loop converged at iteration 95
f(x, y) = 18.23983470863707, grad_norm of hyperfunction= 127.14698119972788
Outer iteration 5
Inner loop converged at iteration 92
f(x, y) = 6.596745179996695, grad_norm of hyperfunction= 95.77394264819782
Outer iteration 6
Inner loop converged at iteration 92
f(x, y) = 0.6448443854897974, grad_norm of hyperfunction= 75.49432363158502
Outer iteration 7
Inner loop converged at iteration 92
f(x, y) = -2.7812355253328604, grad_norm of hyperfunction= 61.46393287876541
Outer iteration 8
Inner loop converged at iteration 92
f(x, y) = -4.937869237625037, g

In [8]:
barrier_algo = BarrierBLO(problem, hparams)

h1_values = [problem.h_1(x_opt_barrier, y_opt_barrier, i) for i in range(problem.num_constraints_h1)]
h2_values = [problem.h_2(x_opt_barrier, y_opt_barrier, i) for i in range(problem.num_constraints_h2)]

y_original_opt_bfbm = barrier_algo.Interior_inner_loop(x_opt_barrier, y_opt_barrier)
print(f"f_theoretical_opt_result={problem.f(x_opt_barrier, y_original_opt_bfbm)}, f_bfbm_numerical_result={problem.f(x_opt_barrier, y_opt_barrier)}")

print(f"g_theoretical_opt_result={problem.g(x_opt_barrier, y_original_opt_bfbm)}, g_bfbm_numerical_result={problem.g(x_opt_barrier, y_opt_barrier)}")

print("constraint violation for randomly generated constraints")
for i, h_val in enumerate(h1_values):
    if h_val >= 0.0:
        print(f"h_1[{i}] = {h_val}")

print("constraint violation for box constraints")
for i, h_val in enumerate(h2_values):
    if h_val >= 0.0:
        print(f"h_2[{i}] = {h_val}")

f_theoretical_opt_result=-15.306563751287975, f_bfbm_numerical_result=-15.153427501682827
g_theoretical_opt_result=-24.423517710938558, g_bfbm_numerical_result=-24.333709631271415
constraint violation for randomly generated constraints
constraint violation for box constraints


# Plot

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(times, grad_norms, marker='o', markersize=1)
plt.xlabel('CPU Time (s)')
plt.ylabel('Gradient Norm')
plt.title('Gradient Norm vs CPU Time')
plt.yscale('log')
plt.grid(True)
plt.show()
